In [1]:
import pandas as pd
import numpy as np
from osgeo import gdal, osr, ogr
import os
import sys
import math

In [2]:
SPATIAL_IDX = 5
TEMPORAL_IDX = 9

In [3]:
'''
Open input file <filename> and return numpy array
'''

def getModisData(filename):
    # This allows gdal to throw python exceptions
    gdal.UseExceptions()
    try:
        modisRaster = gdal.Open(filename)
    except e:
        print("Unable to open tif file")
        print(e)
        print(modisRaster.GetMetadata())
    modisData = modisRaster.ReadAsArray()
    # set NaN values to 0
    modisData = np.where((modisData < 1) | (modisData > 366), 0, modisData)
    modisData = np.floor(modisData)
    return modisData,modisRaster

In [4]:
'''
Set the number of spatial window based on row/col and SPATIAL_IDX
'''
def setSpatialWindow(row,col,arrayDims):
    top = max(0,row-SPATIAL_IDX)
    bottom = min(arrayDims[0],row+SPATIAL_IDX+1)
    left = max(0,col-SPATIAL_IDX)
    right = min(arrayDims[1],col+SPATIAL_IDX+1)
    return top,bottom,left,right

In [5]:
'''
Process the input array determining fire events based on SPATIAL_IDX and TEMPORAL_IDX
'''
def getEventData(inputArray):
    nextId = 1
    eventArray = np.zeros(inputArray.shape) #initialize event array to 0s
    #Traverse pixels in the MODIS Data Array to look for neighboring event pixels that belong to the event
    for row in range(inputArray.shape[0]):
        for col in range(inputArray.shape[1]):
            if inputArray[row,col] : # burn detected
                #print("{},{} burn detected".format(row,col))
                burnDay = inputArray[row,col]
                #print("Next ID={}, burnDay={}:".format(nextId,burnDay))
                # set spatial input and temporal bounds and create spatial windows
                top,bottom,left,right = setSpatialWindow(row,col,inputArray.shape)
                inputWindow = inputArray[top:bottom,left:right]
                temporalMask = np.logical_and(inputWindow>0,abs(inputWindow-burnDay)<=TEMPORAL_IDX)
                eventWindow = eventArray[top:bottom,left:right]
                
                currId = eventArray[row,col] # Event ID for current pixel
                overlapMask = np.logical_and(np.logical_and(eventWindow>0,temporalMask),eventWindow!=currId)
                if overlapMask.any():
                    #true overlap
                    overlapId = np.amin(eventWindow[overlapMask],axis=None)
                    eventWindow[temporalMask] = overlapId
                    if currId != 0:
                        #print("Event merge: setting {} to {}".format(currId,overlapId))
                        eventArray[eventArray==currId] = overlapId
                elif currId == 0: # no overlap and not set so get next ID
                    eventWindow[temporalMask] = nextId
                    nextId += 1
                else: 
                    eventWindow[temporalMask] = currId # propagate current id
    print(eventArray)
    return eventArray

In [6]:
'''
Save the event data to GTiff file
'''
def saveEventData(eventData,input_raster,outfile):
    rows = input_raster.RasterYSize
    cols = input_raster.RasterXSize
    output_raster = gdal.GetDriverByName('GTiff').Create(outfile, cols, rows, 1 ,gdal.GDT_Float32)
    output_raster.SetGeoTransform(input_raster.GetGeoTransform())
    output_raster.SetProjection(input_raster.GetProjection())
    output_raster.GetRasterBand(1).WriteArray(eventData) 

In [8]:
input_path = '../data/MCD64A1/C6/yearly_composites/'
out_path = '../data/MCD64A1/C6/yearly_events/'
ends = ['.tif', '_ms.tif']

for i in range(2001,2018) :
    for j in ends :
        if not os.path.exists(out_path + 'USA_burnevents_' + str(i) + j):
            print(out_path + 'USA_burnevents_' + str(i) + j)

../data/MCD64A1/C6/yearly_events/USA_burnevents_2011_ms.tif
